In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import hashlib
from tensorflow.python.ops import math_ops

from tensorflow.metrics import accuracy

tf.enable_eager_execution()

BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

In [ ]:
from run_nq import input_fn_builder
seq_length=512

name_to_features = {
  "input_ids": tf.FixedLenFeature([], tf.int64),
  "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
  "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
  "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
  "start_bytes": tf.FixedLenFeature([seq_length], tf.int64),
  "end_bytes": tf.FixedLenFeature([seq_length], tf.int64),
}
name_to_features["start_positions"] = tf.FixedLenFeature([], tf.int64)
name_to_features["end_positions"] = tf.FixedLenFeature([], tf.int64)

def _decode_record(record):
  """Decodes a record to a TensorFlow example."""
  example = tf.parse_single_example(record, name_to_features)
  return example

In [ ]:
import os
bert_data_dir = '/Users/deniz/natural_questions/data/'
_dev_path = os.path.join(bert_data_dir, 'dev')
_train_path = os.path.join(bert_data_dir, 'train')
train_files = [os.path.join(_train_path, _file) for _file in os.listdir(_train_path) if _file.endswith(".tf_record")]

train_input_fn = input_fn_builder(
  input_files=train_files,
  seq_length=512,
  is_training=True,
  mode='train')

In [ ]:
import tensorflow as tf
dt = tf.data.TFRecordDataset(train_files)
dt = dt.map(_decode_record, num_parallel_calls=10)
dt = dt.shuffle(buffer_size=100)
dt = dt.batch(32)
it = dt.make_one_shot_iterator()
a = it.get_next()
a.keys()

In [2]:
# context
a = tf.constant([[2,10],[3,20],[4,50]])
a = tf.constant([[0.2,0.5,0.3]])
a = tf.expand_dims(a,1)
print(a.shape)
b = tf.constant([[1,2],[10,20],[1,2]])
b = tf.constant([[1,2]])
b = tf.expand_dims(b,2)
print(b.shape)
c = a + b
out = c.numpy()

(1, 1, 3)
(1, 2, 1)


InvalidArgumentError: cannot compute Add as input #0(zero-based) was expected to be a int32 tensor but is a float tensor [Op:Add] name: add/

In [69]:
a = tf.constant([[0.2, 0.5, 0.3],[0.8, 0.05, 0.15],[0.3, 0.4, 0.3]])
b = tf.constant([[0.1, 0.8, 0.1],[0.2, 0.5, 0.3],[0.3, 0.35, 0.35]])
# a = tf.constant([[0.2, 0.5, 0.3]])
# b = tf.constant([[0.1, 0.8, 0.1]])

In [70]:
start_logits = tf.expand_dims(a, 1)
end_logits = tf.expand_dims(b,-1)
logits = start_logits * end_logits
logits

<tf.Tensor: id=467, shape=(3, 3, 3), dtype=float32, numpy=
array([[[0.02      , 0.05      , 0.03      ],
        [0.16000001, 0.4       , 0.24000001],
        [0.02      , 0.05      , 0.03      ]],

       [[0.16000001, 0.01      , 0.03      ],
        [0.4       , 0.025     , 0.075     ],
        [0.24000001, 0.015     , 0.045     ]],

       [[0.09      , 0.12      , 0.09      ],
        [0.105     , 0.14      , 0.105     ],
        [0.105     , 0.14      , 0.105     ]]], dtype=float32)>

In [71]:
def argmax_2d(start_logits, end_logits):
  start_logits = tf.expand_dims(start_logits, 1)
  end_logits = tf.expand_dims(end_logits,-1)
  logits = start_logits * end_logits
  flat_logits = tf.reshape(logits, shape=[tf.shape(logits)[0], -1])
  _argmax = tf.cast(tf.argmax(flat_logits, axis=-1), dtype=tf.int32)
  return _argmax % tf.shape(logits)[1], _argmax // tf.shape(logits)[2]

In [73]:
argmax_2d(a,b)

(<tf.Tensor: id=518, shape=(3,), dtype=int32, numpy=array([1, 0, 1], dtype=int32)>,
 <tf.Tensor: id=524, shape=(3,), dtype=int32, numpy=array([1, 1, 1], dtype=int32)>)

In [48]:
4 % 3

1

In [32]:
#np.argmax(d,axis=[0,1])
d.argmax(axis=[0, 1])

TypeError: 'list' object cannot be interpreted as an integer

In [13]:
def argmax_2d(tensor):

  # input format: BxHxWxD
  assert rank(tensor) == 4

  # flatten the Tensor along the height and width axes
  flat_tensor = tf.reshape(tensor, (tf.shape(tensor)[0], -1, tf.shape(tensor)[3]))

  # argmax of the flat tensor
  argmax = tf.cast(tf.argmax(flat_tensor, axis=1), tf.int32)

  # convert indexes into 2D coordinates
  argmax_x = argmax // tf.shape(tensor)[2]
  argmax_y = argmax % tf.shape(tensor)[2]

  # stack and return 2D coordinates
  return tf.stack((argmax_x, argmax_y), axis=1)

def rank(tensor):

  # return the rank of a Tensor
  return len(tensor.get_shape())

In [17]:
argmax_2d(c)

<tf.Tensor: id=84, shape=(1, 2, 1), dtype=int32, numpy=
array([[[0],
        [1]]], dtype=int32)>

In [12]:
def batch_gather(tensor, indices):
  """Gather in batch from a tensor of arbitrary size.

  In pseudocode this module will produce the following:
  output[i] = tf.gather(tensor[i], indices[i])

  Args:
    tensor: Tensor of arbitrary size.
    indices: Vector of indices.
  Returns:
    output: A tensor of gathered values.
  """
  shape = get_shape(tensor)
  flat_first = tf.reshape(tensor, [shape[0] * shape[1]] + shape[2:])
  indices = tf.convert_to_tensor(indices)
  offset_shape = [shape[0]] + [1] * (indices.shape.ndims - 1)
  offset = tf.reshape(tf.range(shape[0]) * shape[1], offset_shape)
  output = tf.gather(flat_first, indices + offset)
  return output

In [ ]:
tf.ar

In [15]:
#########################
#####accuracy metric#####
#########################

tf.reset_default_graph()
init = tf.global_variables_initializer()

In [19]:
from tensorflow import metrics

# Start training
with tf.Session() as sess:
    sess.run(init)
    start_ix = tf.expand_dims(tf.constant([10,20,30,40,50]),1)
    end_ix =  tf.expand_dims(tf.constant([10,20,30,40,50]),1)
    start_positions = tf.expand_dims(tf.constant([10,20,30,40,50]),1)
    end_positions = tf.expand_dims(tf.constant([10,20,30,40,60]),1) #80% accuracy

    y_pred = tf.concat([start_ix, end_ix], axis=-1) #[batch_size, 2]
    y_true = tf.concat([start_positions, end_positions], axis=-1) #[batch_size, 2]
    acc = tf.reduce_all(math_ops.equal(y_true, y_pred), axis=-1)
    is_correct = math_ops.to_float(acc)
    a,b = metrics.mean(is_correct)
    
    
    running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES)
    running_vars_initializer = tf.variables_initializer(var_list=running_vars)
    
    sess.run(running_vars_initializer)
    
    # initial op
    a_out = sess.run(a)
    # update op
    b_out = sess.run(b)

In [20]:
a_out, b_out

(0.0, 0.8)

In [ ]:
#from preprocessing.preprocessing import *
from run_nq import *

In [ ]:
bert_data_dir = "/data/nq/natural_questions/v1.0/sample_train"

In [ ]:

train_files = [_file for _file in os.listdir(bert_data_dir) if _file.endswith(".tf_record")]
_file_path = [os.path.join(bert_data_dir, _file) for _file in train_files]
[tf.gfile.MakeDirs(_dir) for _dir in [_train_path, _dev_path]]
print(_file_path)
train_input_fn = input_fn_builder(
    input_file=_file_path,
    seq_length=512,
    is_training=True,
    drop_remainder=True)

In [ ]:
params = {}
params['batch_size'] = 32
_iter = train_input_fn(params)

In [ ]:
_iter = _iter.make_one_shot_iterator()

In [ ]:
out = _iter.get_next()
out.keys()

In [ ]:
def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                  tokenization_info["do_lower_case"]])

    return bert.tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=do_lower_case)

In [ ]:
tokenizer = create_tokenizer_from_hub_module()

In [ ]:
len(dt)

In [ ]:
################################################

In [ ]:
dt = data[-110]
dt.keys()

In [ ]:
dt['question_text']

In [ ]:
#dt['document_tokens'][:10]

In [ ]:
dt['annotations']

In [ ]:
outputs = convert_examples_to_features(dt, tokenizer, 512)

In [ ]:
# answer
if outputs:
    print(outputs[0].targets)

In [ ]:
{i: t for i, t in enumerate(outputs[0].tokens) if i >= outputs[0].targets[0][0] and i <= outputs[0].targets[0][1]}

In [ ]:
short_answer_start = dt['annotations'][0]['short_answers'][0]['start_byte']
short_answer_end = dt['annotations'][0]['short_answers'][0]['end_byte']
[t for t in dt['document_tokens'] if t['start_byte'] >= short_answer_start and t['end_byte'] <= short_answer_end]

In [ ]:
dt['annotations']

In [ ]:
long_answer_start = dt['annotations'][0]['long_answer']['start_byte']
long_answer_end = dt['annotations'][0]['long_answer']['end_byte']
end = dt['annotations'][0]['long_answer']['end_token']+1
start = dt['annotations'][0]['long_answer']['start_token']
dt['document_tokens'][start:end]

In [ ]:
dt = data[0]

In [ ]:
dt.keys()

In [ ]:
dt['document_tokens'][:100]

In [ ]:
#set([t['token'] for dt in data for t in dt['document_tokens'] if t['html_token']])

In [ ]:
from IPython.core.display import display, HTML
display(HTML(dt['document_html']))

In [ ]:
#dt['long_answer_candidates']

In [ ]:
dt['question_tokens']

In [ ]:
dt['annotations'][0]

In [ ]:
end_byte_ix = dt['annotations'][0]['short_answers'][0]['end_byte']
start_byte_ix = dt['annotations'][0]['short_answers'][0]['start_byte']
start_byte_ix, end_byte_ix

In [ ]:
tokens = dt['document_tokens']

In [ ]:
[token for token in tokens if token['start_byte'] >= start_byte_ix and token['end_byte'] <= end_byte_ix]

In [ ]:
end_byte_ix_long = dt['annotations'][0]['long_answer']['end_byte']
start_byte_ix_long = dt['annotations'][0]['long_answer']['start_byte']
start_byte_ix_long, end_byte_ix_long

In [ ]:
#' '.join([token['token'] for token in tokens if token['start_byte'] >= start_byte_ix_long and token['end_byte'] <= end_byte_ix_long and not token['html_token']])

In [ ]:
long_contains_short = 0
short = []
only_long = []
nb_short_answers = []
yes_no_answers = []
multiple_short_answers_q = []
for dt in data:
    if dt['annotations'][0]['yes_no_answer'] != 'NONE':
        yes_no_answers.append(dt)
    short_answers = dt['annotations'][0]['short_answers']
    if short_answers:
        if len(short_answers) > 1:
            multiple_short_answers_q.append(dt)
        
        end_byte_ix = short_answers[0]['end_byte']
        start_byte_ix = short_answers[0]['start_byte']
        end_byte_ix_long = dt['annotations'][0]['long_answer']['end_byte']
        start_byte_ix_long = dt['annotations'][0]['long_answer']['start_byte']
        short.append(dt)  
    else:
        only_long.append(dt) 
print(len(only_long))
print(len(short))
print(len(multiple_short_answers_q))
print(len(yes_no_answers))

In [ ]:
yes_no_answers[0]['annotations']

In [ ]:
multiple_short_answers_q[0]['annotations'][0]['short_answers']

In [ ]:
only_long[0]['annotations']

In [ ]:

#display(HTML(multiple_short_answers_q['document_html']))

In [ ]:
dt['long_answer_candidates'][24]

In [ ]:
dt = only_long[1]
end_byte_ix_long = dt['annotations'][0]['long_answer']['end_byte']
start_byte_ix_long = dt['annotations'][0]['long_answer']['start_byte']
dt['annotations']

In [ ]:
tokens = dt['document_tokens']
' '.join([token['token'] for token in tokens if token['start_byte'] >= start_byte_ix_long and token['end_byte'] <= end_byte_ix_long and not token['html_token']])


In [ ]:
print(dt['question_text'])
#display(HTML(dt['document_html']))